In [109]:
import pandas as pd
import numpy as np
import folium
import requests

# Culinary diversity in New York neighborhoods

In this notebook, we use the Foursquare API to explore restaurants in the neighborhoods of New York city and classify the neighborhoods based on the diversity of their culinary options.

## Geographical data for NY neighborhoods

The geographical coordinates for the neighborhoods in NY are available from: https://geo.nyu.edu/download/file/nyu-2451-34572-geojson.json. We will load this into the variable `geodata`.

In [110]:
geodata = requests.get('https://geo.nyu.edu/download/file/nyu-2451-34572-geojson.json').json()
geodata

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

`geodata['features']` is a list of dictionaries with information for each neighborhood. We will create a dataframe to store the neighborhood name and geographical coordinates. Note the coordinates are given as [longitude, latitude] in the dataset, so we will reverse this to the more conventional [latitude, longitude] using the slice `[::-1]`.

In [111]:
df = pd.DataFrame(columns=['neighborhood', 'coordinates'])

for f in geodata['features']:
    df = df.append({'neighborhood': f['properties']['name'],
               'coordinates': np.array(f['geometry']['coordinates'][::-1])}, ignore_index=True)

In [112]:
df.head()

,neighborhood,coordinates
0,Wakefield,"[40.89470517661, -73.84720052054902]"
1,Co-op City,"[40.87429419303012, -73.82993910812398]"
2,Eastchester,"[40.887555677350775, -73.82780644716412]"
3,Fieldston,"[40.89543742690383, -73.90564259591682]"
4,Riverdale,"[40.890834493891305, -73.9125854610857]"


# Exploring restaurants in neighborhoods

We will now use the Foursquare API to explore these neighborhoods. Below is the information needed to make requests to the API.

In [113]:
CLIENT_ID = 'ESQILQUJ45MEFP5CYAUTQAPRNT1YA22FASZDU0VCI5VIE2EP'
CLIENT_SECRET = 'MZIKSDQUL4OHRIPALRKSUQSKUMMVKM2PCGAMGJPWKJ3G0MBZ'
VERSION = '20200415' # Foursquare API version
LIMIT = 100

In [126]:
lat, lng = df['coordinates'][0]
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?\
                    &client_id={}\
                    &client_secret={}\
                    &v={}\
                    &ll={},{}\
                    &radius={}\
                    &limit={}'.format(
                        CLIENT_ID, 
                        CLIENT_SECRET, 
                        VERSION, 
                        lat, 
                        lng, 
                        radius, 
                        LIMIT)

In [127]:
requests.get(url).json()#["response"]['groups'][0]['items']

{'meta': {'code': 200, 'requestId': '5e978c0402a172001bac8d9e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Wakefield',
  'headerFullLocation': 'Wakefield, Bronx',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 9,
  'suggestedBounds': {'ne': {'lat': 40.899205181110005,
    'lng': -73.84125857127495},
   'sw': {'lat': 40.89020517211, 'lng': -73.8531424698231}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c537892fd2ea593cb077a28',
       'name': 'Lollipops Gelato',
       'location': {'address': '4120 Baychester Ave',
        'crossStreet': 'Edenwald & Bussing Ave',
        'lat': 40.894123150205274,
        'lng': -73.84589162362325,
        'labeledLatLn

# Visualizing neighborhoods

Let's visualize the neighborhoods with a map. To find the geographical center, let's find the average latitude/longitude.

In [115]:
# find average latitude/longitude
lat_avg, lng_avg = df['coordinates'].mean()

In [116]:
map = folium.Map(location=[lat_avg, lng_avg], zoom_start=10)

# loop over neighborhoods
for i in range(df.shape[0]):
    s = df['neighborhood'][i]
    label = folium.Popup(s)

    folium.CircleMarker(
        df['coordinates'][i],
        radius=4,
        popup=label).add_to(map)  
    
map